In [1]:
import argparse
from random import choice
from pathlib import Path
import numpy as np
import os


# torch

import torch
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_

# vision imports

from PIL import Image
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image

# dalle related classes and utils

from dalle_pytorch import OpenAIDiscreteVAE, DiscreteVAE, DALLE
from dalle_pytorch.simple_tokenizer import tokenize, tokenizer, VOCAB_SIZE

from torchvision.datasets import CocoCaptions

In [2]:
gpu_id = '6,7'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device {} : {}".format(gpu_id, use_cuda))

GPU device 6,7 : True


In [3]:
# parser = argparse.ArgumentParser()

# group = parser.add_mutually_exclusive_group(required = False)

# group.add_argument('--vae_path', type = str,
#                     help='path to your trained discrete VAE')

# group.add_argument('--dalle_path', type = str,
#                     help='path to your partially trained DALL-E')

# parser.add_argument('--image_text_folder', type = str, required = True,
#                     help='path to your folder of images and text for learning the DALL-E')
# parser.add_argument('--resume', action='store_true')

class Parser(object):
    def __init__(self, vae_path, dalle_path, image_folder, anno_folder, resume):
        self.vae_path = vae_path
        self.dalle_path = dalle_path
        self.image_folder = image_folder
        self.anno_folder = anno_folder
        self.resume = resume

args = Parser('./vae-final.pt', '', '/vision/7052107/Dalle/coco/train2017/', '/vision/7052107/Dalle/coco/annotations/captions_train2017.json', False)

In [4]:
def exists(val):
    return val is not None

# constants
VAE_PATH = args.vae_path
DALLE_PATH = args.dalle_path
IMAGE_PATH = args.image_folder
ANNO_PATH = args.anno_folder
RESUME = args.resume
epoch_start = 1

EPOCHS = 30
BATCH_SIZE = 30
LEARNING_RATE = 1e-3
GRAD_CLIP_NORM = 0.5

MODEL_DIM = 512
TEXT_SEQ_LEN = 256 # token num
DEPTH = 4
HEADS = 8
DIM_HEAD = 64

In [5]:
if RESUME:
    dalle_path = Path(DALLE_PATH)
    assert dalle_path.exists(), 'DALL-E model file does not exist'

    loaded_obj = torch.load(str(dalle_path))

    dalle_params, vae_params, weights = loaded_obj['hparams'], loaded_obj['vae_params'], loaded_obj['weights']

    vae = DiscreteVAE(**vae_params)
#     print(dalle_params)
#     dalle_params = dict(
#         vae = vae,
#         **dalle_params
#     )

    IMAGE_SIZE = vae_params['image_size']

else:
    if exists(VAE_PATH):
        vae_path = Path(VAE_PATH)
        assert vae_path.exists(), 'VAE model file does not exist'

        loaded_obj = torch.load(str(vae_path))

        vae_params, weights = loaded_obj['hparams'], loaded_obj['weights']

        vae = DiscreteVAE(**vae_params)
        vae.load_state_dict(weights)
    else:
        print('using OpenAIs pretrained VAE for encoding images to tokens')
        vae_params = None

        vae = OpenAIDiscreteVAE()

    IMAGE_SIZE = vae.image_size

    dalle_params = dict(
        vae = vae,
        num_text_tokens = VOCAB_SIZE,
        text_seq_len = TEXT_SEQ_LEN,
        dim = MODEL_DIM,
        depth = DEPTH,
        heads = HEADS,
        dim_head = DIM_HEAD
    )

# helpers

def save_model(path):
    save_obj = {
        'hparams': dalle_params,
        'vae_params': vae_params,
        'weights': dalle.module.state_dict()
    }

    torch.save(save_obj, path)

# create dataset and dataloader

compose = T.Compose([T.Resize(IMAGE_SIZE),
                     T.CenterCrop(IMAGE_SIZE),
                     T.ToTensor(),])

def collate_fn(batch):
    return tuple(zip(*batch))

ds = CocoCaptions(root=IMAGE_PATH, annFile=ANNO_PATH, transform=compose)
dl = DataLoader(ds, BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)

assert len(ds) > 0, 'dataset is empty'
print(f'{len(ds)} image-text pairs found for training')

loading annotations into memory...
Done (t=2.89s)
creating index...
index created!
118287 image-text pairs found for training


In [6]:
# initialize DALL-E

dalle = DALLE(**dalle_params)

if RESUME:
    dalle.load_state_dict(weights)


dalle = torch.nn.DataParallel(dalle).cuda()

# optimizer

opt = Adam(dalle.parameters(), lr = LEARNING_RATE)

# experiment tracker

# import wandb

# wandb.config.depth = DEPTH
# wandb.config.heads = HEADS
# wandb.config.dim_head = DIM_HEAD

# wandb.init(project = 'dalle_train_transformer', resume = RESUME)

# training

In [ ]:
for epoch in range(epoch_start, EPOCHS):
    for i, (images, text) in enumerate(dl):
        images = torch.stack(images)
        text_list = []
        for descriptions in text:
            descriptions = list(filter(lambda t: len(t) > 0, descriptions))
            description= choice(descriptions)
            text_list.append(description)
        text = tokenize(text_list).squeeze(0)
        mask = text != 0
        text, images, mask = map(lambda t: t.cuda(), (text, images, mask))
        loss = dalle(text, images, mask = mask, return_loss = True)
        loss = torch.sum(loss)
        loss.backward()
#         clip_grad_norm_(dalle.parameters(), GRAD_CLIP_NORM)

        opt.step()
        opt.zero_grad()

        log = {}

        if i % 10 == 0:
            print(epoch, i, f'loss - {loss.item()}')

#             log = {
#                 **log,
#                 'epoch': epoch,
#                 'iter': i,
#                 'loss': loss.item()
#             }

        if i % 10 == 0:
            sample_text = text[:1]
            token_list = sample_text.masked_select(sample_text != 0).tolist()
            decoded_text = tokenizer.decode(token_list)

            image = dalle.module.generate_images(
                text[:1],
                mask = mask[:1],
                filter_thres = 0.5   # topk sampling at 0.9
            )

            save_model(f'./dalle_coco.pt')
#             wandb.save(f'./dalle.pt')

#             log = {
#                 **log,
#                 'image': wandb.Image(image, caption = decoded_text)
#             }

#         wandb.log(log)

save_model(f'./dalle-final_coco.pt')
# wandb.save('./dalle-final.pt')
# wandb.finish()


/home/anaconda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


1 0 loss - 18.287561416625977
1 10 loss - 2.4109811782836914
1 20 loss - 2.108689308166504
1 30 loss - 2.3646788597106934
